# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_1.json`):

In [2]:
{
  "name": "Example LCIA Method",
    "version": "1.0",
    "description": "Example LCIA method for greenhouse gas emissions",
    "unit": "kg CO2e",
  "exchanges": [
    {
      "supplier": {
        "name": "Carbon dioxide",
        "operator": "startswith",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "type": "process"
      },
      "value": 1.0
    },
    {
      "supplier": {
        "name": "Methane, fossil",
        "operator": "contains",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "type": "process"
      },
      "value": 28.0
    },
    {
      "supplier": {
        "name": "Dinitrogen monoxide",
        "operator": "equals",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "type": "process"
      },
      "value": 265.0
    }
  ]
}

{'name': 'Example LCIA Method',
 'version': '1.0',
 'description': 'Example LCIA method for greenhouse gas emissions',
 'unit': 'kg CO2e',
 'exchanges': [{'supplier': {'name': 'Carbon dioxide',
    'operator': 'startswith',
    'matrix': 'biosphere'},
   'consumer': {'matrix': 'technosphere', 'type': 'process'},
   'value': 1.0},
  {'supplier': {'name': 'Methane, fossil',
    'operator': 'contains',
    'matrix': 'biosphere'},
   'consumer': {'matrix': 'technosphere', 'type': 'process'},
   'value': 28.0},
  {'supplier': {'name': 'Dinitrogen monoxide',
    'operator': 'equals',
    'matrix': 'biosphere'},
   'consumer': {'matrix': 'technosphere', 'type': 'process'},
   'value': 265.0}]}

In [3]:
# activate the bw project
bw2data.projects.set_current("ecoinvent-3.10.1-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'transport, pipeline, onshore, long distance, natural gas' (ton kilometer, RO, None)

In [5]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_1.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/Users/romain/micromamba/envs/ab/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  4.33it/s]

12790 processed biosphere edges
0 processed technosphere edges


0.029431761560977928

We can print some statistics:

In [6]:
LCA.statistics()

+----------------------+----------------------------------------------+
|       Activity       | transport, pipeline, onshore, long distance, |
|                      |                 natural gas                  |
|     Method name      |              ('some', 'method')              |
|      Data file       |                lcia_example_1                |
|    CFs in method     |                      3                       |
|       CFs used       |                      3                       |
|  Exc. characterized  |                    12790                     |
| Exc. uncharacterized |                    323128                    |
+----------------------+----------------------------------------------+


### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [7]:
df = LCA.generate_cf_table()

In [8]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, from soil or biomass stock","(air,)",feldspar production,feldspar,RoW,9.128448e-16,1.0,9.128448e-16
1,"Carbon dioxide, from soil or biomass stock","(air,)",burnt shale production,burnt shale,DE,1.896211e-15,1.0,1.896211e-15
2,"Carbon dioxide, from soil or biomass stock","(air,)",liquid packaging board production,bark,RER,1.017369e-15,1.0,1.017369e-15
3,"Carbon dioxide, from soil or biomass stock","(air,)","sodium perborate production, tetrahydrate, powder","sodium perborate, tetrahydrate, powder",RoW,8.744544e-17,1.0,8.744544e-17
4,"Carbon dioxide, from soil or biomass stock","(air,)","sodium perborate production, tetrahydrate, powder","sodium perborate, tetrahydrate, powder",RER,9.083129e-19,1.0,9.083129e-19
...,...,...,...,...,...,...,...,...
12785,Dinitrogen monoxide,"(air,)","kerosene production, petroleum refinery operation",kerosene,BR,4.313550e-13,265.0,1.143091e-10
12786,Dinitrogen monoxide,"(air,)","transport, freight, lorry 7.5-16 metric ton, E...","transport, freight, lorry 7.5-16 metric ton, E...",RER,1.775533e-12,265.0,4.705163e-10
12787,Dinitrogen monoxide,"(air,)","market for electricity, high voltage","electricity, high voltage",NE,2.733758e-13,265.0,7.244458e-11
12788,Dinitrogen monoxide,"(air,)","fibreboard production, hard","residual wood, dry",RoW,2.204355e-17,265.0,5.841539e-15


In [8]:
df.groupby("supplier name")["CF"].mean()

supplier name
Carbon dioxide, fossil                               1.0
Carbon dioxide, from soil or biomass stock           1.0
Carbon dioxide, in air                               1.0
Carbon dioxide, non-fossil                           1.0
Carbon dioxide, non-fossil, resource correction      1.0
Carbon dioxide, to soil or biomass stock             1.0
Dinitrogen monoxide                                265.0
Methane, fossil                                     28.0
Name: CF, dtype: float64